In [1]:
#confirm java is running
!java -version

openjdk version "11.0.28" 2025-07-15
OpenJDK Runtime Environment (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)


In [2]:
#install pyspark
!pip install pyspark

In [3]:
#import relevant packages
import os
import json

from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, LongType, TimestampType
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler, StringIndexer, VectorAssembler, VectorIndexer, OneHotEncoder
from pyspark.ml import pipeline
from pyspark.ml.linalg import DenseVector

import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from tabulate import tabulate
import gc
from sklearn.datasets import load_iris
import csv

In [4]:
#start a session
spark = SparkSession.builder.appName("Basics").getOrCreate()

In [5]:
spark.sparkContext.setLogLevel('INFO')

In [6]:
spark.version

'3.5.1'

#Transformations and Actions

##Using map, flatMap and reduceByKey

In [48]:
#map is used in a one to one context; apply a function to each element of an RDD and return a single element

#return the square from 1 to 20:
rdd = spark.sparkContext.parallelize(range(1,21))

rdd.map(lambda x : x * x).collect()

[1,
 4,
 9,
 16,
 25,
 36,
 49,
 64,
 81,
 100,
 121,
 144,
 169,
 196,
 225,
 256,
 289,
 324,
 361,
 400]

In [52]:
#assume you have a linear relationship between x and y, for each increase of unit 1 for x, y increases by 5:

linear_rdd = spark.sparkContext.parallelize(range(1,6))
linear_rdd.map(lambda x : x*5).collect()

[5, 10, 15, 20, 25]

In [4]:
#flatMap will return multiple outputs to one input (a 1 to many relationship). Once used for word count programs.

#first, create a text file from a piece of text
#the below job description was taken from https://ca.indeed.com/viewjob?jk=0478f9262fd81a1f&tk=1j32golobi9dc8j4&from=serp&vjs=3

jobd_text_file = """
Come make your difference in communities across Canada, where authenticity, trust and making connections is valued – as we shape the future of Canadian retail, together. Our unique position as one of the country's largest employers, coupled with our commitment to positively impact the lives of all Canadians, provides our colleagues a range of opportunities and experiences to help Canadians Live Life Well®.

At Companies Limited, we succeed through collaboration and commitment and set a high bar for ourselves and those around us. Whether you are just starting your career, re-entering the workforce, or looking for a new job, this is where you belong.

Does working with some of Canada’s most versatile minds in innovation supporting retail, digital consumer solutions and analytical platforms excite you? Company Technology & Analytics powers game-changing retail solutions, giving our customers the ability to live their lives well.

Come work with a team that values diverse ideas, prioritizes a culture of inclusion and develops our talent from within. Company Technology & Analytics gives you the chance to excel and helps you to strive for success in a big way. Keep reading to learn more!

Article Analyst, Data Operations, (Fixed Term contract)

We have an exciting opportunity for an analyst to join our Data Operation Article team. You will play an integral role in our article setup process, engaging with business partners and participating in projects and initiatives. This position implements processes and governance to ensure data integrity and data standards. The Master Data Analyst will have a primary master data element for which they will be responsible for as well as a secondary element to maintain.

What You'll Do:

Support & implement established data standards and guidelines that relates to Enterprise Data Policies and Procedures in Article Creation and Article Maintenance
Provide store support and conduct work in a timely manner
Proactively supervise data accuracy and facilitate required data corrective actions with business owners, like Merchandising Team, Supply Chain, Costing, Vendors and other internal groups
Contribute to ideas and actions towards continuous improvement of processes within Master Data and Data Quality
Partner in resolving master data discrepancies, SLA (Service Level Agreement) impacts, or other operational challenges affected by master data issues
Participate in day-to-day data cleansing activities.
Build, modify and improve new and ongoing data reports
Maintain good communication with business partners regarding master data requests. Proactively pursue all information required for the completion of work assignments
What You Bring:

Post-Secondary education in Business or related field, or equivalent related experience
Store level retail experience is an asset
Must have intermediate to advance Microsoft Office skills
Strong written and oral communication skills are required
Strong analytical and systems thinking skills
Ability to work independently and proactively
Must be detail oriented and highly organized
Ability to work in teams, influence others and manage conflict
Ability to work based upon specific timelines and dependencies
Experience/knowledge with SAP and SQL/Python/R/SAS/Panda/Jupyter is an asset
What Company Offers You

We offer flexibility and balance, and an environment that sets you up for success no matter where your workspace is located.

Here, you will find a phenomenal team to help you achieve your goals as you help us achieve ours! Work in our fast-paced, exciting Technology environment, helping our stores, colleagues and customers every day.

Company colleagues also enjoy:

On-site Basketball & Volleyball courts,, Dry Cleaning services (1PCC Office)
Paid Vacation
If you’re up to the challenge, then we would love to hear from you. Apply today, and get the process started.

At Company, we respect the environment, source products with integrity and make a positive difference in the community. Our CORE Values – Care, Ownership, Respect and Excellence – guide all our decision-making and come to life through our Blue Culture. We offer our colleagues progressive careers, comprehensive training, flexibility, and other competitive benefits – these are some of the many reasons why we are one of Canada’s Top Employers.

Company recognizes Canada's diversity as a source of national pride and strength. We have made it a priority to reflect our nation’s evolving diversity in the products we sell, the people we hire, and the culture we create in our organization. At Company, we celebrate diversity where differences are valued and supported. Commitment to being an equal opportunity employer is a priority to us, and we encourage people from all backgrounds and identities to apply to our jobs.

Accommodation in the recruitment, assessment, and hiring process is available upon request for applicants with disabilities.

We thank all candidates for their interest but please note, those candidates who meet the minimum requirements for the position will be contacted.

Our commitment to Sustainability and Social Impact is an essential part of the way we do business, and we focus our attention on areas where we can have the greatest impact. Our approach to sustainability and social impact is based on three pillars – Environment, Sourcing and Community – and we are constantly looking for ways to demonstrate leadership in these important areas. Our CORE Values – Care, Ownership, Respect and Excellence – guide all our decision-making and come to life through our Blue Culture. We offer our colleagues progressive careers, comprehensive training, flexibility, and other competitive benefits – these are some of the many reasons why we are one of Canada’s Top Employers, Canada’s Best Diversity Employers, Canada’s Greenest Employers & Canada’s Top Employers for Young People.

If you are unsure whether your experience matches every requirement above, we encourage you to apply anyway. We are looking for varied perspectives which include diverse experiences that we can add to our team.

We have a long-standing focus on diversity, equity and inclusion because we know it will make our company a better place to work and shop. We are committed to creating accessible environments for our colleagues, candidates and customers. Requests for accommodation due to a disability (which may be visible or invisible, temporary or permanent) can be made at any stage of application and employment. We encourage candidates to make their accommodation needs known so that we can provide equitable opportunities.

Please Note:
Candidates who are 18 years or older are required to complete a criminal background check. Details will be provided through the application process.
"""

In [5]:
#export the text file to csv to work within it in spark

with open('job_text_file.txt', 'w', encoding ='unicode-escape') as F:
  F.write(jobd_text_file)

In [6]:
#build a spark session

spark = SparkSession.builder\
          .master('local')\
          .appName('Word_Count')\
          .getOrCreate()

In [7]:
#start the word count app

sc = spark.sparkContext

#read the text file
text_file = sc.textFile('/content/job_text_file.txt')
counts = text_file.flatMap(lambda line: line.split(" ")) \
                            .map(lambda word: (word,1))\
                            .reduceByKey(lambda x, y: x + y)

output = counts.collect()

In [6]:
#count, print the number of words in the text file

for word, count in output[:20]:
  print(f'{word}: {count}')

print('truncated to first 20 observations')

\nCome: 1
make: 4
your: 5
difference: 2
in: 17
communities: 1
across: 1
Canada,: 1
where: 5
authenticity,: 1
trust: 1
and: 55
making: 1
connections: 1
is: 9
valued: 2
\u2013: 9
as: 6
we: 19
shape: 1
truncated to first 20 observations


In [7]:
#let's identify the top 10 most common words

top10 = sorted(output, key=lambda x : x[1], reverse=True)[:10]

for word, n in top10:
  print(f'{word} : {n}')

and : 55
to : 35
the : 22
our : 21
we : 19
a : 18
in : 17
for : 16
of : 15
are : 13


In [8]:
#let's export this app into a script which can be executed in a terminal at later times.

%%writefile wordcount.py
from pyspark.sql import SparkSession

if __name__ == "__main__":
  spark = SparkSession.builder\
          .master('local')\
          .appName('Word_Count')\
          .getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")

  text_file = sc.textFile('/content/job_text_file.txt')

  counts = text_file.flatMap(lambda line: line.split()) \
                            .map(lambda word: (word,1))\
                            .reduceByKey(lambda x, y: x + y)
  output = counts.collect()

  df = spark.createDataFrame(counts, ['word', 'count'])
  df.coalesce(1).write.mode("overwrite").csv("/content/wordcount_csv", header = True)

  spark.stop()

Writing wordcount.py


In the terminal, run spark-submit and the file path

ex: spark-submit /content/wordcount.py

The program will execute and the output file will appear in the same folder as the file path.

## Using filter, union, zip, join, distinct, intersect, subtract, etc

In [15]:
#spart a spark session

spark = (SparkSession.builder
         .appName('Transformations')
         .master('local')
         .getOrCreate())

In [17]:
#a simple UNION ALL of two RDDs

firstRdd = spark.sparkContext.parallelize(range(1,6))
secondRdd = spark.sparkContext.parallelize(range(6,11))
firstRdd.union(secondRdd).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [22]:
#finding the intersection between the two RDDs

firstRdd = spark.sparkContext.parallelize(range(1,9))
secondRdd = spark.sparkContext.parallelize(range(6,11))
firstRdd.intersection(secondRdd).collect()

[6, 8, 7]

In [23]:
#consider a list of employees and their salaries

employeeRDD = spark.sparkContext.parallelize([('Bob'), ('Ashley'), ('Russel'), ('Jessica'),
                                              ('Yusef'), ('Ramona'), ('Lenore'), ('Arash')])
salaryRDD = spark.sparkContext.parallelize([55000,78000,66000,52000,
                                            61000,88000,68000,35000])
#we can join via zip
employeeRDD.zip(salaryRDD).collect()

[('Bob', 55000),
 ('Ashley', 78000),
 ('Russel', 66000),
 ('Jessica', 52000),
 ('Yusef', 61000),
 ('Ramona', 88000),
 ('Lenore', 68000),
 ('Arash', 35000)]

In [31]:
#let's do some salary filtering

emp_sal = employeeRDD.zip(salaryRDD)

#salaries over 75k

salary75k = emp_sal.filter(lambda x : x[1] >= 75000)
salary75k.collect()

[('Ashley', 78000), ('Ramona', 88000)]

In [32]:
#salaries less than 55000

salary55k = emp_sal.filter(lambda x : x[1] <= 55000)
salary55k.collect()

[('Bob', 55000), ('Jessica', 52000), ('Arash', 35000)]

In [36]:
#who has the highest salary?

max_sal = emp_sal.sortBy(lambda x : -x[1])
max_sal.collect()

[('Ramona', 88000),
 ('Ashley', 78000),
 ('Lenore', 68000),
 ('Russel', 66000),
 ('Yusef', 61000),
 ('Bob', 55000),
 ('Jessica', 52000),
 ('Arash', 35000)]

In [38]:
#consider we have four patients and eight medicines in a double-blind study, what are the possible combinations?

patientsRDD = spark.sparkContext.parallelize([('Yusef'), ('Ramona'), ('Lenore'), ('Arash')])
medsRDD = spark.sparkContext.parallelize([('A'), ('B'), ('C'), ('D'),('E'), ('F'), ('G'), ('H')])
patientsRDD.cartesian(medsRDD).collect()

[('Yusef', 'A'),
 ('Yusef', 'B'),
 ('Yusef', 'C'),
 ('Yusef', 'D'),
 ('Yusef', 'E'),
 ('Yusef', 'F'),
 ('Yusef', 'G'),
 ('Yusef', 'H'),
 ('Ramona', 'A'),
 ('Ramona', 'B'),
 ('Ramona', 'C'),
 ('Ramona', 'D'),
 ('Ramona', 'E'),
 ('Ramona', 'F'),
 ('Ramona', 'G'),
 ('Ramona', 'H'),
 ('Lenore', 'A'),
 ('Lenore', 'B'),
 ('Lenore', 'C'),
 ('Lenore', 'D'),
 ('Lenore', 'E'),
 ('Lenore', 'F'),
 ('Lenore', 'G'),
 ('Lenore', 'H'),
 ('Arash', 'A'),
 ('Arash', 'B'),
 ('Arash', 'C'),
 ('Arash', 'D'),
 ('Arash', 'E'),
 ('Arash', 'F'),
 ('Arash', 'G'),
 ('Arash', 'H')]

# Machine Learning & Working With Iris Dataset

In [ ]:
#build a spark session
spark = (SparkSession.builder
         .appName('Spark Machine Learning')
         .config('spark.executor.memory', '1G')
         .config('spark.executor.cores', '4')
         .getOrCreate())

In [ ]:
#load the data & export to CSV
iris = load_iris()

iris_df = pd.DataFrame(
    data = iris.data,
    columns = iris.feature_names
)

iris_df['species'] = iris.target

iris_df['species_name'] = iris_df['species'].apply(lambda x: iris.target_names[x])

iris_df.to_csv("iris.csv", index = False)

In [ ]:
#load the data for spark
url = '/content/iris.csv'

data = spark.read.format('csv')\
      .option("header", "true")\
      .option("inferSchema", "true")\
      .load(url)

data.cache()

DataFrame[sepal length (cm): double, sepal width (cm): double, petal length (cm): double, petal width (cm): double, species: int, species_name: string]

In [ ]:
#tally observations
data.count()

150

In [ ]:
#check the schema
data.printSchema()

root
 |-- sepal length (cm): double (nullable = true)
 |-- sepal width (cm): double (nullable = true)
 |-- petal length (cm): double (nullable = true)
 |-- petal width (cm): double (nullable = true)
 |-- species: integer (nullable = true)
 |-- species_name: string (nullable = true)



In [ ]:
data.show(5)

+-----------------+----------------+-----------------+----------------+-------+------------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|species|species_name|
+-----------------+----------------+-----------------+----------------+-------+------------+
|              5.1|             3.5|              1.4|             0.2|      0|      setosa|
|              4.9|             3.0|              1.4|             0.2|      0|      setosa|
|              4.7|             3.2|              1.3|             0.2|      0|      setosa|
|              4.6|             3.1|              1.5|             0.2|      0|      setosa|
|              5.0|             3.6|              1.4|             0.2|      0|      setosa|
+-----------------+----------------+-----------------+----------------+-------+------------+
only showing top 5 rows



In [ ]:
#tally by species
data.groupBy('species').count().show()

+-------+-----+
|species|count|
+-------+-----+
|      1|   50|
|      2|   50|
|      0|   50|
+-------+-----+



In [ ]:
#summary statistics of the data
data.describe().show()

+-------+------------------+-------------------+------------------+------------------+------------------+------------+
|summary| sepal length (cm)|   sepal width (cm)| petal length (cm)|  petal width (cm)|           species|species_name|
+-------+------------------+-------------------+------------------+------------------+------------------+------------+
|  count|               150|                150|               150|               150|               150|         150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|               1.0|        NULL|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|0.8192319205190406|        NULL|
|    min|               4.3|                2.0|               1.0|               0.1|                 0|      setosa|
|    max|               7.9|                4.4|               6.9|               2.5|                 2|   virginica|
+-------+------------------+-------------------+

In [ ]:
#the data was loaded with species already converted to numerics
#the following showcases use of StringIndexer to make a new column

SIndexer = StringIndexer(inputCol='species', outputCol='species_index')
data_sample = SIndexer.fit(data).transform(data)

data_sample.show(5)

+-----------------+----------------+-----------------+----------------+-------+------------+-------------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|species|species_name|species_index|
+-----------------+----------------+-----------------+----------------+-------+------------+-------------+
|              5.1|             3.5|              1.4|             0.2|      0|      setosa|          0.0|
|              4.9|             3.0|              1.4|             0.2|      0|      setosa|          0.0|
|              4.7|             3.2|              1.3|             0.2|      0|      setosa|          0.0|
|              4.6|             3.1|              1.5|             0.2|      0|      setosa|          0.0|
|              5.0|             3.6|              1.4|             0.2|      0|      setosa|          0.0|
+-----------------+----------------+-----------------+----------------+-------+------------+-------------+
only showing top 5 rows



In [ ]:
#confirm column names
data.columns

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)',
 'species',
 'species_name']

In [ ]:
#remove flower names
df = data.select("species", "sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)")
df.show(5)

+-------+-----------------+----------------+-----------------+----------------+
|species|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|
+-------+-----------------+----------------+-----------------+----------------+
|      0|              5.1|             3.5|              1.4|             0.2|
|      0|              4.9|             3.0|              1.4|             0.2|
|      0|              4.7|             3.2|              1.3|             0.2|
|      0|              4.6|             3.1|              1.5|             0.2|
|      0|              5.0|             3.6|              1.4|             0.2|
+-------+-----------------+----------------+-----------------+----------------+
only showing top 5 rows



In [ ]:
#define the species column as a dense vector (e.g. label) for the future models

input_data = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

In [ ]:
#create a dataframe holding the labels and features
df_index = spark.createDataFrame(input_data, ['label', 'features'])

In [ ]:
df_index.show(5)

+-----+-----------------+
|label|         features|
+-----+-----------------+
|    0|[5.1,3.5,1.4,0.2]|
|    0|[4.9,3.0,1.4,0.2]|
|    0|[4.7,3.2,1.3,0.2]|
|    0|[4.6,3.1,1.5,0.2]|
|    0|[5.0,3.6,1.4,0.2]|
+-----+-----------------+
only showing top 5 rows



In [ ]:
#scaling to normalize the data

stdScaler = StandardScaler(inputCol = "features", outputCol = "features_scaled")

scaler = stdScaler.fit(df_index)

df_scaler = scaler.transform(df_index)

df_scaler.show(5)

+-----+-----------------+--------------------+
|label|         features|     features_scaled|
+-----+-----------------+--------------------+
|    0|[5.1,3.5,1.4,0.2]|[6.15892840883878...|
|    0|[4.9,3.0,1.4,0.2]|[5.9174018045706,...|
|    0|[4.7,3.2,1.3,0.2]|[5.67587520030241...|
|    0|[4.6,3.1,1.5,0.2]|[5.55511189816831...|
|    0|[5.0,3.6,1.4,0.2]|[6.03816510670469...|
+-----+-----------------+--------------------+
only showing top 5 rows



In [ ]:
df_scaled = df_scaler.drop("features")

In [ ]:
#train / test split the model

train_data, test_data = df_scaled.randomSplit([0.8, 0.2], seed = 123)

In [ ]:
train_data.show(5)

+-----+--------------------+
|label|     features_scaled|
+-----+--------------------+
|    0|[5.19282199176603...|
|    0|[5.31358529390013...|
|    0|[5.31358529390013...|
|    0|[5.43434859603422...|
|    0|[5.55511189816831...|
+-----+--------------------+
only showing top 5 rows



## Machine Learning

In [ ]:
#build ML models

model = ['Decision Tree', 'Random Forest', 'Naive Bayes']
model_results = []

In [ ]:
#decision tree

decision_tree = DecisionTreeClassifier(labelCol = "label", featuresCol = "features_scaled")

tree_model = decision_tree.fit(train_data)

tree_predict = tree_model.transform(test_data)

#evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol = "label", predictionCol = "prediction", metricName = "accuracy")

model_accuracy = evaluator.evaluate(tree_predict)

model_results.extend([[model[0], '{:.2}'.format(model_accuracy)]])

In [ ]:
#random forest

random_forest = RandomForestClassifier(labelCol = "label", featuresCol="features_scaled")

forest_model = random_forest.fit(train_data)

forest_predict = forest_model.transform(test_data)

#evaluate the model
rf_evaluator = MulticlassClassificationEvaluator(labelCol = 'label', predictionCol = 'prediction', metricName = 'accuracy')

rf_model_accuracy = rf_evaluator.evaluate(forest_predict)

model_results.extend([[model[1], '{:.2}'.format(rf_model_accuracy)]])

In [ ]:
#Naive Bayes

nv_bayes = NaiveBayes(smoothing = 1.0, modelType = "multinomial", labelCol = "label", featuresCol="features_scaled")

nbayes_model = nv_bayes.fit(train_data)

nbayes_predict = nbayes_model.transform(test_data)

#evaluate model

nv_evaluation = MulticlassClassificationEvaluator(labelCol = "label", predictionCol = 'prediction', metricName = "accuracy")

nv_model_accuracy = nv_evaluation.evaluate(nbayes_predict)

model_results.extend([[model[2], '{:.2}'.format(nv_model_accuracy)]])

In [ ]:
gc.collect()

867

In [ ]:
print(tabulate(model_results, headers=['Classifier Models', 'Accuracy']))

Classifier Models      Accuracy
-------------------  ----------
Decision Tree              0.93
Random Forest              0.97
Naive Bayes                0.97


# Working with Big Data

In [ ]:
#we'll download a large dataset of Amazon reviews using curl

!curl -O https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/review_categories/Automotive.jsonl.gz
!gunzip -f Automotive.jsonl.gz

In [ ]:
#the data is zipped, how big is it?

print(f'The size of the zipped file is: {os.path.getsize("/content/Automotive.jsonl") / (1024 ** 3):.2f} GB')

The size of the zipped file is: 8.13 GB


In [ ]:
#start a spark session

spark = SparkSession.builder.appName("AutomotiveData").getOrCreate()

print(f'The Spark Version is {spark.version}')

The Spark Version is 3.5.1


In [ ]:
#let's try reading the data via pandas

test_df = pd.read_json('/content/Automotive.jsonl', lines = True)
print(test_df)

In [ ]:
#if you run the above code, the session will fail due to all available RAM being used.

#clearly, the data is too massive to be handled with conventional packages.

#Spark is required!

In [ ]:
#let's confirm the columns of the data

with open('/content/Automotive.jsonl', 'r') as file:
  line = file.readline()
  record = json.loads(line)
  print(pd.Series(record))

rating                                                             5.0
title                          It fit our 2012 Chevy Colorado perfect!
text                 Item came as described! It fit our 2012 Chevy ...
images                                                              []
asin                                                        B01LZA8SGZ
parent_asin                                                 B0BV88374L
user_id                                   AGXVBIUFLFGMVLATYXHJYL4A5Q7Q
timestamp                                                1513092936205
helpful_vote                                                         0
verified_purchase                                                 True
dtype: object


In [ ]:
#making a PySpark Dataframe

schema = StructType([
    StructField("rating", DoubleType(), True),
    StructField("title", StringType(), True),
    StructField("text", StringType(), True),
    StructField("images", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("parent_asin", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("helpful_vote", IntegerType(), True),
    StructField("verified_purchase", IntegerType(), True)
])

In [ ]:
spark_df = spark.read.schema(schema).json('/content/Automotive.jsonl')

In [ ]:
print(spark_df.show(5))

+------+--------------------+--------------------+------+----------+-----------+--------------------+--------------------+------------+-----------------+
|rating|               title|                text|images|      asin|parent_asin|             user_id|           timestamp|helpful_vote|verified_purchase|
+------+--------------------+--------------------+------+----------+-----------+--------------------+--------------------+------------+-----------------+
|   5.0|It fit our 2012 C...|Item came as desc...|    []|B01LZA8SGZ| B0BV88374L|AGXVBIUFLFGMVLATY...|+49918-01-03 17:2...|           0|             NULL|
|   5.0|    Easy to put on!!|Ease of applicati...|    []|B0B2WGS5ND| B0B2WGS5ND|AFE337D2J37YRU5U6...|+54545-07-22 13:5...|           0|             NULL|
|   5.0|             Perfect|Nice quality, fra...|    []|B00A0GV20Q| B00A0GV20Q|AEVWAM3YWN5URJVJI...|+45975-02-05 15:4...|           0|             NULL|
|   2.0|      Not waterproof|Description said ...|    []|B08C27WWVG| B08C27W

In [ ]:
spark_df.printSchema()

root
 |-- rating: double (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- images: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- helpful_vote: integer (nullable = true)
 |-- verified_purchase: integer (nullable = true)



In [ ]:
#depending on resources, the summary statistics make take over 10min to compute
spark_df.describe().show()

+-------+------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+
|summary|            rating|               title|        text|              images|                asin|         parent_asin|             user_id|      helpful_vote|vertified_purchase|
+-------+------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+
|  count|          19955450|            19955450|    19955450|            19955450|            19955450|            19955450|            19955450|          19955450|                 0|
|   mean| 4.182790716320604|                 NaN|    Infinity|                NULL| 4.609848654133604E9| 4.615182888455246E9|                NULL|0.6502186119581368|              NULL|
| stddev|1.3815796733887575|                 NaN|         NaN|             

In [ ]:
spark_df.select("title").show(5)

+--------------------+
|               title|
+--------------------+
|It fit our 2012 C...|
|    Easy to put on!!|
|             Perfect|
|      Not waterproof|
|           Very nice|
+--------------------+
only showing top 5 rows



In [ ]:
spark_df.filter(spark_df["verified_purchase"].isNull()).count()

19955450

In [ ]:
spark_df.select(F.countDistinct("verified_purchase")).show()

+---------------------------------+
|count(DISTINCT verified_purchase)|
+---------------------------------+
|                                0|
+---------------------------------+



In [ ]:
spark_df.select("verified_purchase").distinct().show()

+-----------------+
|verified_purchase|
+-----------------+
|             NULL|
+-----------------+



In [ ]:
spark_df.groupBy('rating').count().show()

+------+--------+
|rating|   count|
+------+--------+
|   1.0| 2302427|
|   4.0| 2172554|
|   3.0| 1185429|
|   2.0|  851553|
|   5.0|13443487|
+------+--------+



## Spark SQL with Amazon Data

In [ ]:
#let's run some SQL statements

spark_df.createOrReplaceGlobalTempView("spark_df")

In [ ]:
spark.sql("SELECT rating, helpful_vote \
          FROM global_temp.spark_df \
          WHERE rating >= 4 AND helpful_vote >= 10").show()

+------+------------+
|rating|helpful_vote|
+------+------------+
|   5.0|          11|
|   5.0|          54|
|   5.0|          12|
|   5.0|          83|
|   4.0|          12|
|   4.0|          15|
|   5.0|          13|
|   5.0|          17|
|   4.0|          57|
|   4.0|          14|
|   5.0|         135|
|   4.0|          36|
|   5.0|          10|
|   5.0|          21|
|   5.0|          13|
|   5.0|          11|
|   5.0|          14|
|   5.0|         175|
|   5.0|          31|
|   5.0|          11|
+------+------------+
only showing top 20 rows



In [ ]:
spark.sql("""SELECT title, timestamp, helpful_vote, verified_purchase \
            FROM global_temp.spark_df \
            WHERE verified_purchase IS NULL \
            AND rating <= 2""").show()

+-----+---------+------------+------------------+
|title|timestamp|helpful_vote|vertified_purchase|
+-----+---------+------------+------------------+
+-----+---------+------------+------------------+



In [ ]:
spark.sql("""SELECT title, timestamp, helpful_vote, verified_purchase \
            FROM global_temp.spark_df \
            WHERE verified_purchase = '1' \
            LIMIT 10
            """).show()

+-----+---------+------------+------------------+
|title|timestamp|helpful_vote|vertified_purchase|
+-----+---------+------------+------------------+
+-----+---------+------------+------------------+



In [ ]:
spark.sql("""
          SELECT rating, max(helpful_vote) \
          FROM global_temp.spark_df \
          GROUP BY rating \
          ORDER BY max(helpful_vote) \
          LIMIT 10
          """).show()

+------+-----------------+
|rating|max(helpful_vote)|
+------+-----------------+
|   2.0|             1514|
|   4.0|             1846|
|   3.0|             2036|
|   1.0|             2134|
|   5.0|             2822|
+------+-----------------+

